In [1]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import QtWidgets, QtCore

import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
class cnn_model:
    def __init__(self, path, weights_path):
        self.path = path
        self.weights_path = weights_path
    
    def create_model(self):
        model = Sequential()
        model.add(InputLayer(input_shape = [32,32,1]))
        model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=5, padding='same'))

        model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=5, padding='same'))

        model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same', activation='relu'))
        model.add(MaxPool2D(pool_size=5, padding='same'))

        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(rate=0.5))
        model.add(Dense(10, activation='softmax'))
        optimizer = Adam(lr=1e-3)

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        return model
    
    def predict_image(self):
        model = self.create_model()
        model.load_weights(self.weights_path)
        
        ### helper function
        def one_hot_to_text(predict):
            if np.argmax(predict) == 0:
                predict_label = 'airplane'
            elif np.argmax(predict) == 1:
                predict_label = 'automobile'
            elif np.argmax(predict) == 2:
                predict_label = 'bird'
            elif np.argmax(predict) == 3:
                predict_label = 'cat'
            elif np.argmax(predict) == 4:
                predict_label = 'deer'
            elif np.argmax(predict) == 5:
                predict_label = 'dog'
            elif np.argmax(predict) == 6:
                predict_label = 'frog'
            elif np.argmax(predict) == 7:
                predict_label = 'horse'
            elif np.argmax(predict) == 8:
                predict_label = 'ship'
            else:
                predict_label = 'truck'
        
            return predict_label
        
        import cv2
        img = cv2.imread(self.path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (32,32))
        data = img.reshape(1,32,32,1)
        predict = model.predict([data])
        label = one_hot_to_text(predict)
        return label

In [3]:
class photo_demo(QtWidgets.QWidget):
    #signal
    _signal = QtCore.pyqtSignal()
    def __init__(self,parent = None):
        super(photo_demo,self).__init__(parent)
        vertical_layout = QVBoxLayout()
        grid_layout = QGridLayout()
        grid_layout.setSpacing(10)
        
        # set the size of window
        self.resize(300,300)
                
        self.button = QPushButton()
        self.button.clicked.connect(self.photo_load)
        self.button.setText('UPLOAD\nIMAGE')
        grid_layout.addWidget(self.button,1,0)
        
        self.label = QLabel()
        grid_layout.addWidget(self.label,1,1)
        
        self.label_2 = QLabel()
        self.label_2.setText('The image you \nselected must\n' + 
                             'be one of the\nfollowing types:\n' +  
                             'bird, horse, dog,\n' + 'ship, airplane,\n' + 
                             'truck, frog, deer,\n' + 'cat, automobile.\n')
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)

        grid_layout.addWidget(self.label_2,4,0)
        
        self.label_3 = QLabel()
        grid_layout.addWidget(self.label_3,4,1)
        
        
        self.setWindowTitle('Simple Image Recognition')
        self.setLayout(grid_layout)
        
        #signal
        self._signal.connect(self.mysignal)
    
    def photo_load(self):
        print('Loading..')
       
        fname, _ = QFileDialog.getOpenFileName(self,
                                               'Upload An Image',
                                               '',
                                               'Image files(*.jpg *.jpeg *.png)')
        cnn = cnn_model(fname, '../data/model/model.h5')
        
        self.label.setPixmap(QPixmap(fname))
        self.label_3.setText(f'our best guess is \n{cnn.predict_image()}')
        self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        self.label_3.setStyleSheet("background-color:yellow;")
        
        font = QFont();
        font.setPointSize(30);
        font.setBold(True);
        self.label_3.setFont(font);
        
        self._signal.emit()
        
    def mysignal(self, parameter=1):
        pass

In [4]:
if __name__ == '__main__':
# must construct a QApplication before QWidget
    app = QApplication(sys.argv) 
    photoload = photo_demo()
    photoload.show()
    sys.exit(app.exec_())

Loading..
Loading..
Loading..
Loading..
Loading..
Loading..
Loading..
Loading..
Loading..
Loading..


SystemExit: 0

/Users/Nelson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
